In [ ]:
!pip install tensorflow==2.15.0 tensorflow-addons
!pip install timm


ERROR: Could not find a version that satisfies the requirement tensorflow==2.15.0 (from versions: 2.16.0rc0, 2.16.1, 2.16.2, 2.17.0rc0, 2.17.0rc1, 2.17.0, 2.17.1, 2.18.0rc0, 2.18.0rc1, 2.18.0rc2, 2.18.0, 2.18.1, 2.19.0rc0, 2.19.0, 2.19.1, 2.20.0rc0, 2.20.0)
ERROR: No matching distribution found for tensorflow==2.15.0


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving archive (2).zip to archive (2).zip


In [ ]:
!file archive(2).zip


/bin/bash: -c: line 1: syntax error near unexpected token `('
/bin/bash: -c: line 1: `file archive(2).zip'


In [ ]:
import zipfile
import os

zip_name = list(uploaded.keys())[0]   # auto detect filename

extract_dir = "/content/coco128/"

with zipfile.ZipFile(zip_name, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print("Extracted to:", extract_dir)


Extracted to: /content/coco128/


In [ ]:
import os

print("Root:", os.listdir("/content/"))
print("coco128:", os.listdir("/content/coco128"))
print("Inside coco128:", [f for f in os.listdir("/content/coco128") if os.path.isdir('/content/coco128/' + f)])


Root: ['.config', 'coco128', 'archive (2).zip', 'sample_data']
coco128: ['coco128']
Inside coco128: ['coco128']


In [ ]:
image_dir = "/content/coco128/coco128/images/train2017"
label_dir = "/content/coco128/coco128/labels/train2017"

print("Images:", len(os.listdir(image_dir)))
print("Labels:", len(os.listdir(label_dir)))


Images: 128
Labels: 128


In [ ]:
import os

# COCO classes for YOLO128
coco_classes = [
    'person','bicycle','car','motorcycle','airplane','bus','train','truck','boat',
    'traffic light','fire hydrant','stop sign','parking meter','bench','bird','cat','dog','horse',
    'sheep','cow','elephant','bear','zebra','giraffe','backpack','umbrella','handbag','tie','suitcase',
    'frisbee','skis','snowboard','sports ball','kite','baseball bat','baseball glove','skateboard','surfboard',
    'tennis racket','bottle','wine glass','cup','fork','knife','spoon','bowl','banana','apple','sandwich',
    'orange','broccoli','carrot','hot dog','pizza','donut','cake','chair','couch','potted plant',
    'bed','dining table','toilet','tv','laptop','mouse','remote','keyboard','cell phone','microwave',
    'oven','toaster','sink','refrigerator','book','clock','vase','scissors','teddy bear','hair drier','toothbrush'
]

captions_dict = {}

for file in sorted(os.listdir(label_dir)):
    img_id = file.replace(".txt","")
    full_path = os.path.join(label_dir, file)

    with open(full_path, "r") as f:
        lines = f.readlines()

    classes = {coco_classes[int(l.split()[0])] for l in lines}

    if len(classes) == 0:
        caption = "an image with no objects"
    else:
        caption = "a photo containing " + ", ".join(list(classes))

    captions_dict[img_id] = [caption]

print("Total captions:", len(captions_dict))


Total captions: 128


In [ ]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import numpy as np
import os

# Correct dataset paths
image_dir = "/content/coco128/coco128/images/train2017"
label_dir = "/content/coco128/coco128/labels/train2017"

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

# Load pretrained ResNet18
resnet = models.resnet18(pretrained=True)
resnet = torch.nn.Sequential(*list(resnet.children())[:-1])  # remove final FC
resnet.to(device)
resnet.eval()

# Preprocessing
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

def extract_feature(path):
    img = Image.open(path).convert("RGB")
    x = transform(img).unsqueeze(0).to(device)
    with torch.no_grad():
        feat = resnet(x)
    return feat.cpu().numpy().reshape(-1)


Using device: cuda
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
100%|██████████| 44.7M/44.7M [00:00<00:00, 193MB/s]


In [ ]:
# Read image IDs
image_files = sorted(os.listdir(image_dir))
image_ids = [f.replace(".jpg","") for f in image_files]

train_ids = image_ids[:100]   # first 100 images used for training
test_ids  = image_ids[100:128]

# Extract features
features = {}
for img_id in train_ids:
    img_path = os.path.join(image_dir, img_id + ".jpg")
    features[img_id] = extract_feature(img_path)

feat_dim = len(list(features.values())[0])
feat_dim


512

In [ ]:
# COCO128 class names
coco_classes = [
    'person','bicycle','car','motorcycle','airplane','bus','train','truck','boat',
    'traffic light','fire hydrant','stop sign','parking meter','bench','bird','cat','dog','horse',
    'sheep','cow','elephant','bear','zebra','giraffe','backpack','umbrella','handbag','tie','suitcase',
    'frisbee','skis','snowboard','sports ball','kite','baseball bat','baseball glove','skateboard','surfboard',
    'tennis racket','bottle','wine glass','cup','fork','knife','spoon','bowl','banana','apple','sandwich',
    'orange','broccoli','carrot','hot dog','pizza','donut','cake','chair','couch','potted plant',
    'bed','dining table','toilet','tv','laptop','mouse','remote','keyboard','cell phone','microwave',
    'oven','toaster','sink','refrigerator','book','clock','vase','scissors','teddy bear','hair drier','toothbrush'
]

captions_dict = {}

label_files = sorted(os.listdir(label_dir))

for file in label_files:
    img_id = file.replace(".txt","")

    with open(os.path.join(label_dir, file), "r") as f:
        lines = f.readlines()

    classes = {coco_classes[int(l.split()[0])] for l in lines}

    if len(classes) == 0:
        caption = "an image with no objects"
    else:
        caption = "a photo containing " + ", ".join(list(classes))

    captions_dict[img_id] = [caption]

print("Total caption entries:", len(captions_dict))


Total caption entries: 128


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Keep only image IDs that have labels
train_ids = [i for i in train_ids if i in captions_dict]
test_ids  = [i for i in test_ids  if i in captions_dict]

# Build training caption list
train_captions = [
    "startseq " + captions_dict[i][0].lower() + " endseq"
    for i in train_ids
]

tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_captions)

vocab_size = len(tokenizer.word_index) + 1
sequences = tokenizer.texts_to_sequences(train_captions)
max_length = max(len(s) for s in sequences)

vocab_size, max_length


(84, 18)

In [ ]:
print("Sample image filenames:", sorted(os.listdir(image_dir))[:10])
print("Sample label filenames:", sorted(os.listdir(label_dir))[:10])


Sample image filenames: ['000000000009.jpg', '000000000025.jpg', '000000000030.jpg', '000000000034.jpg', '000000000036.jpg', '000000000042.jpg', '000000000049.jpg', '000000000061.jpg', '000000000064.jpg', '000000000071.jpg']
Sample label filenames: ['000000000009.txt', '000000000025.txt', '000000000030.txt', '000000000034.txt', '000000000036.txt', '000000000042.txt', '000000000049.txt', '000000000061.txt', '000000000064.txt', '000000000071.txt']


In [ ]:
import numpy as np
import tensorflow as tf

X1, X2, y = [], [], []

for img_id in train_ids:
    caption = "startseq " + captions_dict[img_id][0].lower() + " endseq"
    seq = tokenizer.texts_to_sequences([caption])[0]

    for i in range(1, len(seq)):
        in_seq = seq[:i]
        out_word = seq[i]

        in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
        out_word = tf.keras.utils.to_categorical(out_word, num_classes=vocab_size)

        X1.append(features[img_id])
        X2.append(in_seq)
        y.append(out_word)

X1 = np.array(X1)
X2 = np.array(X2)
y  = np.array(y)

X1.shape, X2.shape, y.shape


((732, 512), (732, 18), (732, 84))

In [ ]:
from tensorflow.keras.layers import Input, Embedding, Dropout, Dense, SimpleRNN, add
from tensorflow.keras.models import Model

# IMAGE FEATURE ENCODER
inp_img = Input(shape=(feat_dim,))
fe = Dropout(0.5)(inp_img)
fe = Dense(256, activation='relu')(fe)

# CAPTION DECODER
inp_cap = Input(shape=(max_length,))
se = Embedding(vocab_size, 256, mask_zero=True)(inp_cap)
se = Dropout(0.5)(se)
se = SimpleRNN(256)(se)

# MERGE
decoder = add([fe, se])
decoder = Dense(256, activation='relu')(decoder)
output = Dense(vocab_size, activation='softmax')(decoder)

model = Model([inp_img, inp_cap], output)
model.compile(loss='categorical_crossentropy', optimizer='adam')

model.fit([X1, X2], y, epochs=20, batch_size=128)
model.save("simple_rnn_captioning.h5")


Epoch 1/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 8s 489ms/step - loss: 4.1340
Epoch 2/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 3.1667 
Epoch 3/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 2.5703 
Epoch 4/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 2.1689 
Epoch 5/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 1.9771 
Epoch 6/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 1.8872 
Epoch 7/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 1.8091 
Epoch 8/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 1.7086 
Epoch 9/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 1.6672
Epoch 10/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 1.4920
Epoch 11/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 1.4652 
Epoch 12/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 1.3133 
Epoch 13/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 1.2654 
Epoch 14/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 1.1111 
Epoch 15/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 1.1285 
Epoch 16/20
6/6 ━━━━━━━━━━━━━━━━━

In [ ]:
def predict_caption(model, feat):
    text = "startseq"
    for _ in range(max_length):
        seq = tokenizer.texts_to_sequences([text])[0]
        seq = pad_sequences([seq], maxlen=max_length)
        yhat = np.argmax(model.predict([feat.reshape(1,-1), seq], verbose=0))
        word = tokenizer.index_word.get(yhat)
        if word is None or word == "endseq":
            break
        text += " " + word
    return text.replace("startseq","").strip()


In [ ]:
results = {}

for img_id in test_ids[:10]:
    path = os.path.join(image_dir, img_id + ".jpg")
    feat = extract_feature(path)
    results[img_id] = predict_caption(model, feat)

results


{'000000000540': 'a photo containing bird person',
 '000000000542': 'a photo containing traffic person',
 '000000000544': 'a photo containing baseball glove person',
 '000000000560': 'a photo containing cake cake clock',
 '000000000562': 'a photo containing donut',
 '000000000564': 'a photo containing sports ball person',
 '000000000569': 'a photo containing dog person',
 '000000000572': 'a photo containing person',
 '000000000575': 'a photo containing dog person',
 '000000000581': 'a photo containing bird person'}

In [ ]:
for img in results:
    print("IMAGE:", img)
    print("Generated Caption:", results[img])
    print("Ground Truth:", captions_dict[img][0])
    print("-"*50)


IMAGE: 000000000540
Generated Caption: a photo containing bird person
Ground Truth: a photo containing traffic light, bus, truck, car, airplane
--------------------------------------------------
IMAGE: 000000000542
Generated Caption: a photo containing traffic person
Ground Truth: a photo containing handbag, tie, airplane, person
--------------------------------------------------
IMAGE: 000000000544
Generated Caption: a photo containing baseball glove person
Ground Truth: a photo containing baseball glove, sports ball, baseball bat, person
--------------------------------------------------
IMAGE: 000000000560
Generated Caption: a photo containing cake cake clock
Ground Truth: a photo containing cell phone, potted plant, sink, toilet
--------------------------------------------------
IMAGE: 000000000562
Generated Caption: a photo containing donut
Ground Truth: a photo containing toothbrush, cup
--------------------------------------------------
IMAGE: 000000000564
Generated Caption: a p

In [ ]:
for img in results:
    gen = results[img].split()
    if len(gen) < 3:
        print("FAIL CASE:", img)
        print("Generated:", results[img])
        print("GT:", captions_dict[img][0])
        print("Reason: Simple RNN has limited long-term memory → short/generic caption.\n")


In [ ]:
from tensorflow.keras.layers import Input, Embedding, Dropout, Dense, SimpleRNN, add
from tensorflow.keras.models import Model

# IMAGE ENCODER
inp_img = Input(shape=(feat_dim,))
fe = Dropout(0.5)(inp_img)
fe = Dense(256, activation='relu')(fe)    # stay 256-dim

# CAPTION DECODER
inp_cap = Input(shape=(max_length,))
se = Embedding(vocab_size, 256, mask_zero=True)(inp_cap)
se = Dropout(0.5)(se)
se = SimpleRNN(512)(se)                   # RNN = 512 units
se = Dense(256, activation='relu')(se)    # project 512 → 256 for merging

# MERGE
decoder = add([fe, se])
decoder = Dense(256, activation='relu')(decoder)
output = Dense(vocab_size, activation='softmax')(decoder)

model = Model([inp_img, inp_cap], output)
model.compile(loss='categorical_crossentropy', optimizer='adam')

model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3       │ (None, 18)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, 18, 256)   │     21,504 │ input_layer_3[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_2       │ (None, 512)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_3 (Dropout) │ (None, 18, 256)   │          0 │ embedding_1[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal_1         │ (None, 18)        │          0 │ input_layer_3[0]… │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 512)       │          0 │ input_layer_2[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ simple_rnn_1        │ (None, 512)       │    393,728 │ dropout_3[0][0],  │
│ (SimpleRNN)         │                   │            │ not_equal_1[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 256)       │    131,328 │ dropout_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 256)       │    131,328 │ simple_rnn_1[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 256)       │          0 │ dense_3[0][0],    │
│                     │                   │            │ dense_4[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 256)       │     65,792 │ add_1[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 84)        │     21,588 │ dense_5[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 765,268 (2.92 MB)

 Trainable params: 765,268 (2.92 MB)

 Non-trainable params: 0 (0.00 B)